In [196]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go
import plotly.express  as px

'''
df_raw.columns: ['date', ...(other features), target feature]
'''

# data = pd.read_csv(r'datasets\ETT-data\TrainOneDay.csv').iloc[11520:].dropna(axis=0)
# data = data.rename(columns={
#     'DateTime':'date',  'Power':'OT'
# })

# data  = data[['date'] + list(data.columns[2:]) + ['OT']]
# data.to_csv(r'datasets\ETT-data\ETTh1.csv',index=False)

"\ndf_raw.columns: ['date', ...(other features), target feature]\n"

In [197]:
site = pd.read_csv(r'datasets\ETT-data\TC3 POWER 15MIN.csv')
elc = pd.read_csv(r'datasets\ETT-data\TC3 TPC 15MIN.csv')
data = pd.merge(elc,site,how='inner',on='Dtime')
col = ['Dtime'] + [f'd{c}' for c in range(2,37)] + ['Power']
data = data[col]
data = data.rename(columns={
    'Dtime':'date',  'Power':'OT'
})
data = data[data.date > '2022-05'].reset_index(drop=True)
data = data.drop(data.std()[(data.std() == 0)].index, axis=1)
data.to_csv(r'datasets\ETT-data\ETTh1.csv',index=False)

C:\Users\ZME\AppData\Local\Temp\ipykernel_12612\1057534681.py:10: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [198]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.date,y=data.OT,line=dict(width=2)))
fig.add_vrect(
    x0=data.date.iloc[int(data.shape[0]*0.8)],
    x1=data.date.iloc[int(data.shape[0]*0.9)],
    fillcolor="red",
    opacity=0.2,
    line_width=0,
)

fig.add_vrect(
    x0=data.date.iloc[int(data.shape[0]*0.9)],
    x1=data.date.iloc[-1],
    fillcolor="green",
    opacity=0.2,
    line_width=0,
)

In [ ]:
# conda activate SCINET
# cd .\Desktop\SCINet\

In [297]:
!python .\run_ETTh.py --hidden-size 2 --batch_size 16  --seq_len 192 --label_len 96 --pred_len 96 --features MS --model_name h2_b16_2day_ms --evaluate True
!python .\run_ETTh.py --hidden-size 2 --batch_size 32  --seq_len 192 --label_len 96 --pred_len 96 --features MS --model_name h2_b32_2day_ms --evaluate True
!python .\run_ETTh.py --hidden-size 2 --batch_size 16  --seq_len 480 --label_len 96 --pred_len 96 --features MS --model_name h2_b16_5day_ms --evaluate True
!python .\run_ETTh.py --hidden-size 2 --batch_size 32  --seq_len 480 --label_len 96 --pred_len 96 --features MS --model_name h2_b32_5day_ms --evaluate True
!python .\run_ETTh.py --hidden-size 2 --batch_size 16  --seq_len 672 --label_len 96 --pred_len 96 --features MS --model_name h2_b16_7day_ms --evaluate True
!python .\run_ETTh.py --hidden-size 2 --batch_size 32  --seq_len 672 --label_len 96 --pred_len 96 --features MS --model_name h2_b32_7day_ms --evaluate True



@@@@@ 35 35 1
Args in experiment:
Namespace(model='SCINet', data='ETTh1', root_path='./datasets/ETT-data/', data_path='ETTh1.csv', features='MS', target='OT', freq='h', checkpoints='exp/ETT_checkpoints/', inverse=False, embed='timeF', use_gpu=True, gpu=0, use_multi_gpu=False, devices='0', seq_len=192, label_len=96, pred_len=96, concat_len=0, single_step=0, single_step_output_One=0, lastWeight=1.0, cols=None, num_workers=0, itr=0, train_epochs=100, batch_size=16, patience=50, lr=0.003, loss='rmse', lradj=1, use_amp=False, save=True, model_name='h2_b16_2day_ms', resume=False, evaluate=True, hidden_size=2.0, INN=1, kernel=5, dilation=1, window_size=12, dropout=0.5, positionalEcoding=False, groups=1, levels=3, stacks=1, num_decoder_layer=1, RIN=False, decompose=False, enc_in=35, dec_in=35, c_out=1, detail_freq='h')
SCINet(
  (blocks1): EncoderTree(
    (SCINet_Tree): SCINet_Tree(
      (workingblock): LevelSCINet(
        (interact): InteractorLevel(
          (level): Interactor(
        

In [299]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual,interactive
from glob import glob
files = glob(r'exp\ett_results\*')

In [300]:

@interact
def show_articles_more_than(column=files):
    true = np.load(rf'{files[0]}\true_scales.npy')
    pred = np.load(rf'{files[0]}\pred_scales.npy')
    return true,pred
@interact
def show_articles_more_than(column=files, x=(0, true.shape[0]-1, 1)):
    fig = plt.figure(figsize=(20,5))
    plt.ylim([true.min(), true.max()])
    plt.plot(true[x].flatten())
    plt.plot(pred[x].mean(axis=1).flatten())
    return plt.show()

interactive(children=(Dropdown(description='column', options=('exp\\ett_results\\SCINet_ETTh1_ftMS_sl192_ll96_…

In [15]:
import numpy as np
from sklearn.metrics import mean_squared_error


def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100